In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 

import pathlib
import tqdm
import pickle 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


tf.get_logger().setLevel('ERROR')

2023-04-02 23:17:44.920300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
raw_data_train_path = '../input/cs4248lun/fulltrain.csv'
raw_data_test_path = '../input/cs4248lun/balancedtest.csv'

df_train = pd.read_csv(raw_data_train_path, header=None)
df_test = pd.read_csv(raw_data_test_path, header=None)
df_train.head()


,0,1
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [4]:
df_test.head()

,0,1
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


In [5]:
df_train.dropna(axis=0, inplace=True)

X_train= np.array(df_train[1])
y_train = np.array(df_train[0])
X_test= np.array(df_test[1])
y_test= np.array(df_test[0])

enc = OneHotEncoder(sparse=False)

y_train_ohe = enc.fit_transform(y_train.reshape((-1, 1)))
y_test_ohe = enc.fit_transform(y_test.reshape((-1, 1)))

X_train_train, X_train_val, y_train_ohe_train, y_train_ohe_val = train_test_split(X_train, y_train_ohe, test_size=0.2, random_state=123)

print(X_train.shape, y_train_ohe.shape)  
print(X_train_train.shape, y_train_ohe_train.shape)
print(X_train_val.shape, y_train_ohe_val.shape)
print(X_test.shape, y_test_ohe.shape)  


(48854,) (48854, 4)
(39083,) (39083, 4)
(9771,) (9771, 4)
(3000,) (3000, 4)


In [6]:
from collections import Counter
Counter(y_train)

Counter({1: 14047, 2: 6942, 3: 17870, 4: 9995})

In [5]:
fp_prefix = 'v1'
model_code = 'bert_lstm_adamw'


In [14]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [16]:
from tensorflow.keras.optimizers.experimental import AdamW

In [12]:
def build_classifier_model_bert_lstm():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['sequence_output']
    net = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.1))(net),
    net = tf.keras.layers.Dense(4, activation='softmax', name='classifier')(net[0])
    model = tf.keras.Model(text_input, net)

    model.compile(
        optimizer=AdamW(learning_rate=5e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy',]
    )
    
    return model

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback 


In [16]:
early_stopper = EarlyStopping(monitor='val_loss', patience=5)

In [17]:
classifier_model_bert_lstm_es = build_classifier_model_bert_lstm()
history_es = classifier_model_bert_lstm_es.fit(
    x=X_train_train,
    y=y_train_ohe_train, 
    validation_data=(X_train_val, y_train_ohe_val),
    batch_size=256,
    epochs=200, 
    callbacks=[early_stopper], 
    verbose=1,
)
hist_es = pd.DataFrame(history_es.history)
hist_es['epoch'] = history_es.epoch
os.makedirs(f'./models/{model_code}', exist_ok=True)
hist_es.to_json(f'./models/{model_code}/{fp_prefix}_{model_code}_model_es_hist.json')
classifier_model_bert_lstm_es.save(f'./models/{model_code}/{fp_prefix}_{model_code}_model_es.h5')

Epoch 1/200
153/153 [==============================] - 594s 4s/step - loss: 0.2164 - accuracy: 0.9263 - val_loss: 0.0557 - val_accuracy: 0.9805
Epoch 2/200
153/153 [==============================] - 574s 4s/step - loss: 0.0409 - accuracy: 0.9867 - val_loss: 0.0559 - val_accuracy: 0.9806
Epoch 3/200
153/153 [==============================] - 617s 4s/step - loss: 0.0190 - accuracy: 0.9940 - val_loss: 0.0372 - val_accuracy: 0.9888
Epoch 4/200
153/153 [==============================] - 616s 4s/step - loss: 0.0085 - accuracy: 0.9976 - val_loss: 0.0408 - val_accuracy: 0.9881
Epoch 5/200
153/153 [==============================] - 616s 4s/step - loss: 0.0048 - accuracy: 0.9988 - val_loss: 0.0420 - val_accuracy: 0.9886
Epoch 6/200
153/153 [==============================] - 617s 4s/step - loss: 0.0039 - accuracy: 0.9990 - val_loss: 0.0458 - val_accuracy: 0.9877
Epoch 7/200
153/153 [==============================] - 618s 4s/step - loss: 0.0019 - accuracy: 0.9997 - val_loss: 0.0415 - val_accuracy:

In [18]:
classifier_model_bert_lstm = build_classifier_model_bert_lstm()
history = classifier_model_bert_lstm.fit(
    x=X_train,
    y=y_train_ohe, 
    batch_size=256,
    epochs=len(hist_es), 
#     callbacks=[TqdmCallback(verbose=0)], 
    verbose=1,
)
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
os.makedirs(f'./models/{model_code}', exist_ok=True)
hist.to_json(f'./models/{model_code}/{fp_prefix}_{model_code}_model_hist.json')
classifier_model_bert_lstm.save(f'./models/{model_code}/{fp_prefix}_{model_code}_model.h5')
                        

Epoch 1/8
191/191 [==============================] - 613s 3s/step - loss: 0.2037 - accuracy: 0.9290
Epoch 2/8
191/191 [==============================] - 601s 3s/step - loss: 0.0376 - accuracy: 0.9882
Epoch 3/8
191/191 [==============================] - 598s 3s/step - loss: 0.0171 - accuracy: 0.9948
Epoch 4/8
191/191 [==============================] - 597s 3s/step - loss: 0.0073 - accuracy: 0.9982
Epoch 5/8
191/191 [==============================] - 597s 3s/step - loss: 0.0055 - accuracy: 0.9986
Epoch 6/8
191/191 [==============================] - 596s 3s/step - loss: 0.0050 - accuracy: 0.9984
Epoch 7/8
191/191 [==============================] - 597s 3s/step - loss: 0.0022 - accuracy: 0.9995
Epoch 8/8
191/191 [==============================] - 606s 3s/step - loss: 0.0024 - accuracy: 0.9993


In [6]:
from sklearn.metrics import classification_report

In [22]:
y_test_pred_bert_lstm = classifier_model_bert_lstm.predict(X_test)
y_test_pred_bert_lstm = [enc.categories_[0][i] for i in np.argmax(y_test_pred_bert_lstm, axis=-1)]
print(classification_report(y_test, y_test_pred_bert_lstm))

94/94 [==============================] - 11s 122ms/step
              precision    recall  f1-score   support

           1       0.68      0.82      0.74       750
           2       0.82      0.76      0.79       750
           3       0.23      0.06      0.09       750
           4       0.57      0.92      0.70       750

    accuracy                           0.64      3000
   macro avg       0.57      0.64      0.58      3000
weighted avg       0.57      0.64      0.58      3000



Covid-19 Out-of-Domain Test

In [7]:
ood_data_test_path = '../input/cs4248lun/covid_unreliable_news_cleaned_numbered.csv'
df_ood_test = pd.read_csv(ood_data_test_path, header=None)
df_ood_test.head()


,0,1
0,4,It is likely that eventually it will become en...
1,4,Our researchers are committed to ending the CO...
2,4,Could the pandemic of the century have been av...
3,4,"Coronavirus disease 2019, or COVID-19, is a ne..."
4,4,"Children, including very young children, can d..."


In [9]:
X_ood_test= np.array(df_ood_test[1])
y_ood_test= np.array(df_ood_test[0])
y_ood_test_ohe = enc.fit_transform(y_ood_test.reshape((-1, 1)))
print(X_ood_test.shape, y_ood_test_ohe.shape)  


(120,) (120, 4)


In [24]:
classifier_model_bert_lstm = build_classifier_model_bert_lstm()
classifier_model_bert_lstm.load_weights(f'./models/{model_code}/{fp_prefix}_{model_code}_model.h5')
y_ood_test_pred_bert_lstm = classifier_model_bert_lstm.predict(X_ood_test)
y_ood_test_pred_bert_lstm = [enc.categories_[0][i] for i in np.argmax(y_ood_test_pred_bert_lstm, axis=-1)]
print(classification_report(y_ood_test, y_ood_test_pred_bert_lstm))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4/4 [==============================] - 15s 3s/step
              precision    recall  f1-score   support

           1       0.57      1.00      0.73        31
           2       0.25      0.03      0.06        30
           3       0.20      0.07      0.10        29
           4       0.40      0.70      0.51        30

    accuracy                           0.46       120
   macro avg       0.36      0.45      0.35       120
weighted avg       0.36      0.46      0.36       120

